In [1]:
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3

In [2]:
s3_client = boto3.client("s3")
sess = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
key_prefix = "pt_lightning_ddp_tune"

In [9]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz && tar xzf cifar-10-python.tar.gz && rm cifar-10-python.tar.gz

--2022-04-29 18:25:24--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  80.6MB/s    in 2.0s    

2022-04-29 18:25:26 (80.6 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]



In [10]:
cifar_data_path = sess.upload_data("cifar-10-batches-py", bucket, key_prefix=f"{key_prefix}/input_data/cifar-10-batches-py")

In [11]:
cifar_path = "/".join(cifar_data_path.split("/")[:-1])

In [13]:
subnet=None
security_group_ids=None

In [14]:
estimator_gpu_tune_cifar = PyTorch(
    source_dir = "src",
    entry_point="tune_cifar.py",
    subnets=subnet,
    security_group_ids=security_group_ids,
    role=role,
    instance_count=4, 
    instance_type="ml.g5.12xlarge",
    framework_version="1.10",
    py_version="py38",
    hyperparameters={"use-gpu":True, 
                     "num-samples":10, 
                     "num-workers":2, 
                     "num-epochs":20}
)

In [ ]:
estimator_gpu_tune_cifar.fit({"train": cifar_path})

2022-04-29 18:26:59 Starting - Starting the training job...ProfilerReport-1651256818: InProgress
...
2022-04-29 18:27:43 Starting - Preparing the instances for training....